# Building a spelling correction/word suggestion module
**fastText** can be applied to solve a plethora of problems such as **spelling correction, auto
suggestions**, and so on since it is based on **sub-word representation**. Datasets such as **user
search query, chatbots or conversations, reviews**, and **ratings** can be used to **build fastText
models**. We can apply them to **enhance the customer experience** in the future by providing
information such as **better suggestions, displaying better products, autocorrecting user
input**, and so on. 

In this notebook I am using a **fastText model** trained on some **comments data** from **[Kaggle's toxic comment classification challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)** to build a **spelling correction/word suggestion** module. By providing some **incorrect spellings** to the built model, we can determine **how well the model
does in terms of correcting them**.

# fastText
**fastText** is a **word embedding** method that is an **extension of the word2vec model**. Instead of learning vectors for words directly, **fastText** represents each word as an **n-gram of characters**. 

> For example, take the word, ***artificial*** with *n = 3*, the **fastText** representation of this word is 
>
> [***ar, art, rti, tif, ifi, fic, ici, ial, al***]  

This helps capture the **meaning of shorter words** and allows the **embeddings** to understand **suffixes** and **prefixes**. Once the word has been represented using **character n-grams**, a **skip-gram** model is **trained to learn** the **embeddings**. 

> This model is considered to be a **bag of words** model with a **sliding window over a word** because **no internal structure of the word** is taken into account. As long as the **characters are within this window**, the **order of the n-grams doesn’t matter**.

**fastText** works well with **rare words**. So even if a **word wasn’t seen during training**, or in other words, is **not in vocabulary**, it can be **broken down into n-grams to get its embeddings**.

**Word2vec** fails to provide any vector representation for **words** that are **not in the model dictionary**. This is a **huge advantage** of the **fastText** method.

In [ ]:
import pandas as pd
import spacy
from gensim.models import FastText

In [ ]:
# Static variables
TRAIN_CSV = 'https://raw.githubusercontent.com/tianqwang/Toxic-Comment-Classification-Challenge/master/data/train.csv'
MODEL_PATH = '/content/drive/MyDrive/fastText/fast_text.model'
IS_MODEL_SAVE = True
IS_MODEL_LOAD = False

In [ ]:
toxic_df = pd.read_csv(TRAIN_CSV, on_bad_lines='skip')
toxic_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
nlp = spacy.load(
    name='en', 
    disable=["tagger", "parser", "ner"]
)

In [ ]:
def preprocess_corpus(corpus: list, batch_size: int = None) -> list:
  global nlp
  return [
    [
     token.lemma_.casefold() for token in doc 
     if token.is_alpha and not token.is_stop
    ] for doc in nlp.pipe(texts=corpus, batch_size=batch_size)
  ]

In [ ]:
if IS_MODEL_LOAD:
  # Load the model
  fast_text = FastText.load(MODEL_PATH)
  
  # Define new corpus to re-build the vocabulary
  corpus = []
  
  # Update vcabulary
  is_update = bool(corpus)

else:
  # Initialize the model
  fast_text = FastText(
    size=200, # Dimensionality of the word vectors (Default 100)
    window=3, # Sliding window size over a word (Default 5)
    min_count=1, # Min word frequency (Default 5)
    min_n=1, # Min length of char ngrams (Default 3)
    max_n=5, # Max length of char ngrams (Default 6)
    workers=4 # Number of threads to train the model  (Default 3)
  )

  # Define initial corpus to build the model on
  corpus = toxic_df['comment_text']
  
  # Initiate vocabulary
  is_update = False

# Define the documents
documents = preprocess_corpus(corpus)

# Check if there are any documents
if any(bool(doc) for doc in documents):
  
  # Build the vocabulary
  fast_text.build_vocab(sentences=documents, update=is_update)
    
  # Train the model
  fast_text.train(
    sentences=documents,
    total_examples=fast_text.corpus_count,
    epochs=10
  )

In [ ]:
len(fast_text.wv.vocab)

150837

In [ ]:
fast_text.wv.most_similar('eplain', topn=5)

[('xplain', 0.8468446731567383),
 ('explain', 0.7839565873146057),
 ('eexplain', 0.7662200927734375),
 ('plain', 0.7631980180740356),
 ('reexplain', 0.7383497953414917)]

In [ ]:
# Save trained fastText model
if IS_MODEL_SAVE:
  fast_text.save(MODEL_PATH)

# fastText and document distances

In [ ]:
sentences = [
    "Obama speaks to the media in Illinois",
    "President greets the press in Chicago",
    "Apple is my favorite company"
]

In [ ]:
# L2-normalized vectors
fast_text.init_sims(replace=True)
print('L2-normalized WMD between:')
for i in range(len(sentences) - 1):
  for j in range(i + 1, len(sentences)):
    wmd = fast_text.wv.wmdistance(sentences[i], sentences[j])
    print(' - Sentences {} and {}: {:.4f}'.format(i +1, j + 1, wmd))

L2-normalized WMD between:
 - Sentences 1 and 2: 0.3505
 - Sentences 1 and 3: 0.3424
 - Sentences 2 and 3: 0.4762


# Summary
The results that we obtained in the **spelling correction** and **distance calculations** would be
 **potentially better** if **pre-trained fastText models were used** since those are mostly built on **Wikipedia text corpora** and are **more generalized** to understand **different data points**.

**fastText** is a **very convenient technique** for **building word representations** using **character-level** features. It **outperformed Word2Vec** since it incorporated **internal word structure**
 information and associated it with **morphological features**, which are very important in certain languages. It also allows us to **represent words not present in the original vocabulary**. 

This approach based on the **character-level embedding** by incorporating **n-grams of characters** can be further **extended** to build **embeddings for documents** and **sentences** by using an approach called **Sent2Vec**.

Moreover, a **Universal Sentence Encoder**,
which is one of the **most recent algorithms** that's used to **build sentence-level embeddings** could be aplied.